# Search Agent with Web Search (Perplexity's `sonar`) and Ollama

Original [Infotrend's CoreAI](https://github.com/Infotrend-Inc/CoreAI) Demo project: https://github.com/Infotrend-Inc/CoreAI-DemoProjects/tree/main/Agent
**(modified to use Ollama and Perplexity's OpenAI API and only using the code's original "Hybrid" mode).**
We recommend end users study the orignal code to see additional features.

## Description

This notebook demonstrates a comprehensive AI agent system that provides users with full control over model selection and processing approaches. 
The agent uses Perplexity's `sonar` model for web-enhanced responses and various local LLM models (through Ollama) for general processing.

To enhance the functionality of the CoreAI  environment, we need to install some libraries not pre-installed but required for this notebook. 

## Pre-requisites

To support features of this notebook with CoreAI, we need to install some libraries that are not pre-installed but are required. 

**Create and Activate the Virtual Environment:**

Open a terminal within the Jupyter notebook (`File -> New -> Terminal`).
Navigate to this project's folder; where we want to set up the environment (where this notebook is located) and run:

```bash
export PROJECT_NAME="SearchAgent"
export PIP_CACHE_DIR=`pwd`/.cache/pip
mkdir -p $PIP_CACHE_DIR
python -m venv --system-site-packages myvenv
source myvenv/bin/activate
pip install ipykernel
python -m ipykernel install --user --name=${PROJECT_NAME}-myvenv --display-name="Python (${PROJECT_NAME}-myvenv)"
echo ""; echo "Before continuing load the created Python kernel: Python (${PROJECT_NAME}-myvenv)"
```

This will create a local virtual environment to contain installed files to the mounted `/iti` folder (and not modify the container's files).

**Load the Python kernel described above before running the cell below** (it might take a few seconds for the kernel to appear in the list of kernels).

Install the required Libraries (from `requirements.txt`).
The rest of this notebook relies on the proper kernel to be loaded and environment variables to be set. 

In [ ]:
import os

def set_env_with_cache_dir(env_var_name: str, subdir: str):
    base_cache = os.path.join(os.getcwd(), ".cache")
    full_path = os.path.join(base_cache, subdir)
    os.environ[env_var_name] = full_path
    os.makedirs(full_path, exist_ok=True)

set_env_with_cache_dir("PIP_CACHE_DIR", "pip")

In [ ]:
!. ./myvenv/bin/activate && pip install -r requirements.txt

## Environment Configuration

By default, the notebook will hide dot files (i.e. `.env`).
Edit the `env.example` file included in this notebook to reflect used API keys and rename the file `.env` from a "Terminal".

**Security Note**: Never commit API keys to version control.

In [ ]:
# Load environment configuration file
# This sets up the basic structure for API credentials

import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
if 'PERPLEXITY_API_KEY' not in os.environ:
    print("Missing PERPLEXITY_API_KEY -- FIX BEFORE CONTINUING")
print("✓ .env loaded")

## Import Required Libraries
Imports all necessary Python libraries and modules required for the AI agent functionality. The imports include web requests handling, type definitions, environment variable management, datetime utilities, JSON processing, and the core LiteLLM library for model interactions.

In [ ]:
# Import all required libraries
import os
import requests
from typing import List, Dict, Optional, Union
from dotenv import load_dotenv
import datetime
import json
from openai import OpenAI

# Load and validate environment variables
load_dotenv()
PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")

if not PERPLEXITY_API_KEY:
    raise ValueError("Set PERPLEXITY_API_KEY in the .env file.")

from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama' # required but ignored
)

In [ ]:
class UserChoiceAgent:
    """
    A comprehensive AI agent that provides users with full control over model selection
    and processing approaches. Supports both web-enabled Sonar models and regular LLMs.
    """
    
    def __init__(self):
        """Initialize the agent with model discovery and user preference setup."""
        # Initialize model storage
        self.available_models = []
        self.sonar_models = []
        self.regular_models = []
        self.conversation_history = []
        
        # Default user preferences
        self.user_preferences = {
            "default_sonar_model": "sonar",
            "default_llm_model": "llama3.1:8b",
            "preferred_approach": "hybrid"
        }

##  Response Generation Methods
Implements the core response generation functionality for individual model interactions. The method handles the communication with LiteLLM proxy, manages API calls, processes responses, and provides comprehensive error handling for robust model interactions.

In [ ]:
def generate_response(self, prompt: str, model: str) -> Dict:
    """Generate response using specified model."""
    try:
        print(f" Using {model}...")
        
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            stream=False
        )
        
        return {
            "content": response.choices[0].message.content,
            "model": model,
            "type": "llm_response"
        }
        
    except Exception as e:
        return {"error": f"Error with {model}: {str(e)}"}

# Add method to UserChoiceAgent class
UserChoiceAgent.generate_response = generate_response


## Sonar Search Methods
Specializes in web-enabled search functionality using Perplexity's Sonar models. The implementation provides access to current web information, real-time data, and up-to-date content by leveraging Sonar's web search capabilities for enhanced response accuracy.



In [ ]:
def sonar_search(self, query: str, sonar_model: str = None) -> Dict:
    """Use Sonar model for web-enabled search and response generation."""

    perplexity_client = OpenAI(api_key=PERPLEXITY_API_KEY, base_url="https://api.perplexity.ai")
    
    try:
        print(f" Using {sonar_model} for web search...")
        
        response = perplexity_client.chat.completions.create(
            model=sonar_model,
            messages=[{"role": "user", "content": query}],
            stream=False
        )

    except Exception as e:
        return {"error": f"Sonar search failed: {str(e)}"}

    response_dict = {}
    # Convert response to dict using model_dump() for Pydantic models
    try:
        response_dict = response.model_dump()
    except AttributeError:
        # Fallback for objects that don't support model_dump
        response_dict = vars(response)

    response_text = response.choices[0].message.content

    # Add citations if the key is present in the response, irrelevant of the model provider
    citations_text = ""
    if 'citations' in response_dict:
        citations_text += "\n\nCitations:\n"
        for i in range(len(response_dict['citations'])):
            citations_text += f"\n[{i+1}] {response_dict['citations'][i]}\n"
    response_text += citations_text

    return {
        "content": response_text,
        "model": sonar_model,
        "type": "sonar_search"
    }



# Add method to UserChoiceAgent class
UserChoiceAgent.sonar_search = sonar_search


## Hybrid Processing Methods
Implements the hybrid approach that combines web search with LLM processing for comprehensive responses. The method first gathers current information using Sonar models, then enhances and structures the content using regular LLMs for optimal readability and analysis.

In [ ]:
def hybrid_approach(self, query: str, sonar_model: str = None, llm_model: str = None) -> Dict:
    """Combine Sonar web search with LLM processing for comprehensive responses."""
    sonar_model = sonar_model or self.user_preferences["default_sonar_model"]
    llm_model = llm_model or self.user_preferences["default_llm_model"]
    
    if not sonar_model or not llm_model:
        return {"error": "Both Sonar and LLM models required for hybrid mode"}
    
    try:
        print(f" Hybrid Mode: {sonar_model} + {llm_model}")
        
        # Step 1: Get web-grounded information from Sonar
        print("** Step 1: Get web-grounded information from Sonar")
        sonar_result = self.sonar_search(query, sonar_model)
        if "error" in sonar_result:
            return sonar_result
        
        # Step 2: Use LLM to process and enhance the information
        print("** Step 2: Use LLM to process and enhance the information")
        enhancement_prompt = f"""
Based on the following web-grounded information, provide a comprehensive and well-structured answer to the query: "{query}"

Web-grounded information:
{sonar_result['content']}

Please:
When explaining, start with a one-paragraph “Executive Summary” section.
Then add a “Core Concepts” section to explain in simple terms with related ideas on how it works. 
Follow this with a more detailed “Further Information” section, providing a longer description of content from the “Core Concepts” section.
Add a “Related Concepts” section to include important things to know about a subject, but also key limitations, and things to know first.

1. Organize the information clearly
2. Add relevant context or explanations where helpful
3. Maintain accuracy to the source information
4. Structure the response for better readability
"""
        print(f"-----------------------------------PROMPT:\n {enhancement_prompt}")
        llm_result = self.generate_response(enhancement_prompt, llm_model)
        if "error" in llm_result:
            return llm_result
        
        return {
            "content": llm_result['content'],
            "sonar_content": sonar_result['content'],
            "sonar_model": sonar_model,
            "llm_model": llm_model,
            "type": "hybrid_response"
        }
        
    except Exception as e:
        return {"error": f"Hybrid processing failed: {str(e)}"}

# Add method to UserChoiceAgent class
UserChoiceAgent.hybrid_approach = hybrid_approach


## Query Processing Methods
Contains the main query processing logic that routes user requests to appropriate processing approaches. The method supports automatic approach detection, manual selection, and switching between different processing modes based on query characteristics and user preferences.

In [ ]:
def process_query(self, query: str, options: Dict = None) -> Dict:
    """Process query with full user choice options."""
    if not options:
        options = {}
    
    # Use user preferences as defaults
    approach = options.get("approach", self.user_preferences["preferred_approach"])
    sonar_model = options.get("sonar_model", self.user_preferences["default_sonar_model"])
    llm_model = options.get("llm_model", self.user_preferences["default_llm_model"])
    
    # Auto-detect approach if needed
    if approach == "auto":
        search_keywords = ['latest', 'current', 'recent', 'news', 'today', 'update', 'what is happening']
        if any(word in query.lower() for word in search_keywords):
            approach = "sonar_only"
        else:
            approach = "hybrid"
    
    # Execute based on approach
    if approach == "sonar_only":
        result = self.sonar_search(query, sonar_model)
    elif approach == "llm_only":
        result = self.generate_response(query, llm_model)
    elif approach == "hybrid":
        result = self.hybrid_approach(query, sonar_model, llm_model)
    else:
        result = {"error": f"Unknown approach: {approach}"}
    
    # Add to conversation history
    self.conversation_history.append({
        "query": query,
        "result": result,
        "options": options,
        "timestamp": datetime.datetime.now().isoformat()
    })
    
    return result

# Add method to UserChoiceAgent class
UserChoiceAgent.process_query = process_query


## Utility Methods
Provides essential utility functions for preference management and model information retrieval.

In [ ]:
def update_preferences(self, **kwargs):
    """Update user preferences programmatically."""
    for key, value in kwargs.items():
        if key in self.user_preferences:
            self.user_preferences[key] = value
            print(f"✓ Updated {key} to {value}")

def list_models(self) -> Dict[str, List[str]]:
    """Return categorized list of available models and current preferences."""
    return {
        "sonar_models": self.sonar_models,
        "regular_models": self.regular_models,
        "all_models": self.available_models,
        "user_preferences": self.user_preferences
    }

# Add methods to UserChoiceAgent class
UserChoiceAgent.update_preferences = update_preferences
UserChoiceAgent.list_models = list_models


## Agent Initialization

In [ ]:
# Initialize the agent with user choice
agent = UserChoiceAgent()

In [ ]:
user_input = "Perform a websearch on different presentation at the OpenInfra 2025 Europe and discuss topics relevant to Artifical Intelligence and Scientific research" 
result = agent.process_query(user_input)
print(f"\n\n\n ---------------------------------Response:\n{result.get('content', result.get('error', 'No response'))}")

In [ ]:
agent.user_preferences = {
    "default_sonar_model": "sonar",
    "default_llm_model": "gpt-oss:20b",
    "preferred_approach": "hybrid"
}

user_input = "What are the OpenInfra 2025 Europe presenations listed on LinkedIn. Reference and discuss topics relevant to Artifical Intelligence and Scientific research" 
result = agent.process_query(user_input)
print(f"\n\n\n ---------------------------------Response:\n{result.get('content', result.get('error', 'No response'))}")

# With Ollama

The above code was designed during the summer 2025.
At the end of September 2025, Ollama added a "Web search" capability to their API: https://ollama.com/blog/web-search

The following makes uses of it without the various added features of the original Search Agent code: we will use `tools` to perform a query and use a local model to interpret the retrieved results.

In [ ]:
# Load environment configuration file
# This sets up the basic structure for API credentials

import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
if 'OLLAMA_API_KEY' not in os.environ:
    print("Missing OLLAMA_API_KEY -- FIX BEFORE CONTINUING")
print("✓ .env loaded")

In [ ]:
from ollama import chat, web_fetch, web_search

available_tools = {'web_search': web_search, 'web_fetch': web_fetch}

messages = [{'role': 'user', 'content': "In LLM, what is an Agent, what are tool uses"}]

while True:
  response = chat(
    model='gpt-oss:20b',
    messages=messages,
    tools=[web_search, web_fetch],
    think=True
    )
  if response.message.thinking:
    print('Thinking: ', response.message.thinking)
  if response.message.content:
    print('Content: ', response.message.content)
  messages.append(response.message)
  if response.message.tool_calls:
    print('Tool calls: ', response.message.tool_calls)
    for tool_call in response.message.tool_calls:
      function_to_call = available_tools.get(tool_call.function.name)
      if function_to_call:
        args = tool_call.function.arguments
        result = function_to_call(**args)
        print('Result: ', str(result)[:200]+'...')
        # Result is truncated for limited context lengths
        messages.append({'role': 'tool', 'content': str(result)[:2000 * 4], 'tool_name': tool_call.function.name})
      else:
        messages.append({'role': 'tool', 'content': f'Tool {tool_call.function.name} not found', 'tool_name': tool_call.function.name})
  else:
    break